# Basics of Signal Processing
**Authors**: Anmol Parande, Hoang Nguyen, Jordan Grelling

In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt

# I. DFT

## I.a PSD

## I.b Windowing

## I.c Blackman-Tukey

# II. Sampling Theory

## II.a Aliasing

# III. Filtering

## III.a Filtering in the Time Domain

## III.b Filtering in the Frequency Domain

# IV. Spectral Analysis

## IV.a STFT

## IV.b Tradeoffs of the STFT

# Resources

1. [Anmol's Course Notes from EE120 (Signals and Systems)](https://aparande.gitbook.io/berkeley-notes/ee120-0)
2. [Anmol's Course Notes from EE123 (Digital Signal ](https://aparande.gitbook.io/berkeley-notes/ee123-0)